## Note : 
Mean free path : $\lambda = \frac{1}{n \sigma}$

If $n \rightarrow 100 \times n$, then :
- $\lambda \rightarrow \frac{\lambda}{100}$
- $\text{cell size} \rightarrow \frac{\text{cell size}}{100}$ ($100^2$ more cells because the grid is 2D)
- $dt \rightarrow \frac{dt}{100}$

But you can lower, down to a certain extent, the number of particles per cell to "not incease too much the computation time".

## Representative physical length scale

The system has two lenghts : 
* lenght of the tube : $L = 0.01$ $m$
* width : $w = 0.001$ $m$

The width is the representative physical length scale that constrains the flow.

So : $K_d = \frac{\lambda}{w}$.




In [1]:
%matplotlib widget

# notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as scipy
from pathlib import Path

from lppydsmc.data.saver import Saver
from plotting import analysis
from lppydsmc.utils import physics

plt.figure.max_open_warning = False

In [2]:
def convert_to_species(row):
    if(row['species'] == 0 or row['species'] == 0.0):
        return 'I'
    else :
        return 'I-'

## Loading results

In [3]:
dir_path = Path('results/')

simu = 'dmsc_multiple_species' # low, mid, thruster_low, test_hall

dz = 0.001

In [4]:

if(simu == 'dmsc_multiple_species'): # representative lenght scale varies along the flow ...
    dt = 1e-8 # 1e-8
    mr = 3200000 # 16000000 # np.array([80000000,   800000.])
    name = 'test_dmsc_reactions_with_loading_lots.h5'
    lenght = 0.018 # m
    width = 0.005
    cell_size = 0.001
    temperature = np.array([300,1000]) # K
    nb_cells = 41
    volume = cell_size*cell_size*dz*nb_cells

In [5]:
store = pd.HDFStore(dir_path/name)

In [6]:
# defining each variable
collisions_per_cell_full = store['collisions_per_cell'] 
df_full = store['df']
pmax_per_cell = store['pmax_per_cell'] # cell-discretization not taken into account 
averages_per_cell = store['averages_per_cell'] # idem
total_deleted = store['total_deleted']
total_distance = store['total_distance']
total_proba = store['total_proba']
collisions_with_walls = store['collisions_with_walls']
df_out_particles = store['df_out_particles']

In [7]:
np.sum(collisions_per_cell_full)

10177.0

In [8]:
# df_full['x'] = df_full['x'].astype(float)
# df_full['y'] = df_full['y'].astype(float)
# df_full['vx'] = df_full['vx'].astype(float)
# df_full['vy'] = df_full['vy'].astype(float)
# df_full['vz'] = df_full['vz'].astype(float)

In [9]:
choice = 'I'
gamma = 5/3. # roughly for atomes, for diatomic molecule : 7/5.

mass = physics.get_mass_part(53, 53, 74) # I
molecular_mass = 0.1269 # kg/mol
    
verbose=False

In [10]:
sound_vel = physics.speed_of_sound(molecular_mass, temperature, gamma)
print(f'Speed of sound for [{choice}] at temperature {temperature} K : {sound_vel} m/s')

Speed of sound for [I] at temperature [ 300 1000] K : [180.99192586 330.44453506] m/s


## Note : 
To save a figure, you can use : `analysis.save_fig(fig, path, title = None, dpi = 400, figsize = None)`, which allows you to easily save a figure while giving it a title and changing its size.
You can also use the interactive widget.

## Choosing the part of the system to plot 

In [11]:
# grid resolutions for the thruster

thruster = True

if(thruster):
    res_x_0, res_y_0 = 18, 5 # 3 grids
else:
    res_x_0, res_y_0 = 10, 1 # 100, 10 # 25, 10

total_nb_cells = res_x_0*res_y_0

In [12]:
fig, ax = plt.subplots(1, constrained_layout = True, figsize = (res_x_0, res_y_0));

fact = 10
volume_cell_plot = volume/(fact*fact*res_x_0*res_y_0)
duration = df_full.index.unique().values.shape[0]
analysis.hist2d(ax, df_full, bins = (fact*res_x_0,fact*res_y_0), weights = mr*np.ones(df_full.shape[0])/(duration*volume_cell_plot), stat = 'count'); # TODO: add cmap

In [12]:
use_part = False

# tube last part
if(use_part):
    if(thruster):
        part = 1 # 2
        if(part == 1):
            x_min, x_max = 0, 3e-3 
            y_min, y_max = 0.0, 5e-3
            
            # collisions
            res_x, res_y = 3, 5 
            offset = 0
            cells = [k for k in range(15)]
            
        elif(part == 2):
            x_min, x_max = 0.005, 1e-2
            y_min, y_max = 2e-3, 3e-3
            
            # collisions
            res_x, res_y = 5,1
            offset = 27
            cells = [27, 32, 37, 42, 47] 
            
        volume = (x_max-x_min)*(y_max-y_min)*dz
        df = df_full.loc[(df_full['x']>x_min) & (df_full['x']<x_max) & (df_full['y']>y_min) & (df_full['y']<y_max)].copy(deep=True)

        # in addition we want to select the right cells in collisions
        # plotting the mfp for the selected cells.
        # first along x axis, then along y axis
        nb_rows = collisions_per_cell_full.shape[0]
        
        index_collisions = collisions_per_cell_full.index.unique().values
        collisions_per_cell = np.zeros((res_x,res_y,index_collisions.shape[0]))

        for cell in cells :
            i, j = (cell-offset)//res_y_0, (cell-offset)%res_y_0
            collisions_per_cell[i,j,:] = collisions_per_cell_full[cell:nb_rows:total_nb_cells].values
    else: # tube
        hall = False
        if(not hall):
            x_min, x_max = 9e-3, 1e-2
            y_min, y_max = 0, 1e-3

            res_x, res_y = 1, 1
            offset = 9 
            cells = [9]
        else:
            x_min, x_max = 2e-2, 2.5e-2
            y_min, y_max = 0, 1e-2

            res_x, res_y = 5, 10 
            offset = 200 
            cells = [offset+k for k in range(50)] 
            
        volume = (x_max-x_min)*(y_max-y_min)*dz
        df = df_full.loc[(df_full['x']>x_min) & (df_full['x']<x_max) & (df_full['y']>y_min) & (df_full['y']<y_max)].copy(deep=True)
        # in addition we want to select the right cells in collisions
        # plotting the mfp for the selected cells.
        # first along x axis, then along y axis
            
        nb_rows = collisions_per_cell_full.shape[0]
        
        index_collisions = collisions_per_cell_full.index.unique().values
        collisions_per_cell = np.zeros((res_x,res_y,index_collisions.shape[0]))

        for cell in cells :
            i, j = (cell-offset)//res_y_0, (cell-offset)%res_y_0
            collisions_per_cell[i,j,:] = collisions_per_cell_full[cell:nb_rows:total_nb_cells].values
        
else:
    df = df_full # another reference.
    nb_rows = collisions_per_cell_full.shape[0]

    ## we are still
    index_collisions = collisions_per_cell_full.index.unique().values
    collisions_per_cell = np.zeros((res_x_0,res_y_0,index_collisions.shape[0]))
    for cell in range(total_nb_cells):
        i, j = cell//res_y_0, cell%res_y_0
        collisions_per_cell[i,j,:] = collisions_per_cell_full[cell:nb_rows:total_nb_cells].values

## Choosing frames to plot

In [30]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
fraction_transient = 0.20 # in practice, it is closer to 0.2-0.3
# generally speaking, you choose frames so you have the steady state
frames = unique_index[int(fraction_transient*nb_save):nb_save] 

if(verbose):
    print(f'Available frames :  {unique_index}')
    print(f'Max iteration : {iterations}')
    print(f'Choosen frames (for plotting) : \n{frames}')

In [14]:
frames[-1] 

2000

## Particles analysis

In [15]:
# df['species'] = df.apply(convert_to_species, axis = 1)

In [16]:
df_it = df.loc[np.isin(df.index, frames)]

In [17]:
df_it.head()

,x,y,vx,vy,vz,species
5,0.013122,0.002007,213.978347,-18.295125,-228.513556,0.0
5,0.015102,0.002074,162.771286,52.456725,98.139414,0.0
5,0.003266,0.003222,-436.299770,6.608027,275.600839,0.0
5,0.009743,0.002338,61.790534,-88.370322,23.803820,0.0
5,0.002316,0.003996,68.408131,-51.526181,50.390298,0.0


In [18]:
df_it.groupby(df_it["species"]).count()

,x,y,vx,vy,vz
species,,,,,
0.0,106416073,106416073,106416073,106416073,106416073
1.0,16824,16824,16824,16824,16824


# Not mandatory
fig, ax = plt.subplots()
sns.despine(fig)

sns.scatterplot(
    data = df_it,
    # bins = 100,
    x="x", y ="y", hue="species", s = 5.
);

In [19]:
# scatter plot
dy = 1e-3
y_offset = 2e-3 # m

In [20]:
df_central = df_it.loc[(df_it['y']>y_offset) & (df_it['y']<y_offset+dy)]

In [21]:
df_central_I = df_central.loc[df_central['species']==0.0]
df_central_Im = df_central.loc[df_central['species']==1.0]

In [24]:
fig, ax = plt.subplots(constrained_layout = True)
dx_neutrals, bins_neutrals = 1e-4, 180 # should always be equal to 1.8e-2 m
dx_ions, bins_ions = 0.5e-3, 36 # should always be equal to 1.8e-2 m

ax.set_yscale('log')
duration_it = df_it.index.unique().values.shape[0]

analysis.hist1d(ax, df_central_I, 'x', bins = bins_neutrals, weights = mr/(dy*dx_neutrals*dz*duration_it)*np.ones(df_central_I.shape[0]), color = 'b', histtype = 'step', label = 'I')
analysis.hist1d(ax, df_central_Im, 'x', bins = bins_ions, weights = mr/(dy*dx_ions*dz*duration_it)*np.ones(df_central_Im.shape[0]), color = 'r', histtype = 'step', label = 'I-')
analysis.set_axis(ax, 'x', 'density')
ax.legend(loc='best');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plotting density of particles evolution

In [25]:
df_species = df['species']

In [26]:
nb_parts = df_species.groupby(df_species.index).agg('count').values

In [27]:
df_I = df_species.loc[df_species==0.0]
nb_parts_I = df_I.groupby(df_I.index).agg('count').values

In [28]:
fig, ax = plt.subplots(constrained_layout = True)
ax.plot(df.index.unique()*dt, nb_parts_I*mr/volume, label = 'I')
ax.plot(df.index.unique()*dt, (nb_parts-nb_parts_I)*mr/volume, label = 'I-')
ax.set_yscale('log')
plt.legend(loc='best')
analysis.set_axis(ax, x = 'time', y = 'density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Velocity distribution

In [29]:
df_it_I = df_it.loc[df_it['species']==0.0]
df_it_Im = df_it.loc[df_it['species']==1.0]

In [31]:
fig, axes = analysis.velocity_distribution(df_it_Im, frames = None, bins = 100, density = True, sharex = False, sharey = False); # (df_it_I['vx']<1000) & 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
fig, axes = analysis.velocity_distribution(df_it_I.loc[(df_it_I['vx']>-1000)&(df_it_I['vx']<1000)], frames = None, bins = 100, density = True, sharex = False, sharey = False); # (df_it_I['vx']<1000) & 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### stream plot

In [32]:
dp = 1e-4
df['i'] = (df['x']/dp).astype(int)
df['j'] = (df['y']/dp).astype(int)

In [57]:
groups = df.loc[df['species'] == 0.0].groupby(['i','j']).mean()

In [58]:
groups = groups.reset_index()

In [59]:
groups.head()

,i,j,x,y,vx,vy,vz,species
0,0,0,0.000050,0.000050,4.005587,2.153542,1.999296,0.0
1,0,1,0.000051,0.000149,2.710209,4.677024,2.460784,0.0
2,0,2,0.000051,0.000249,13.005699,1.855610,12.562790,0.0
3,0,3,0.000051,0.000350,12.782944,4.534887,7.706892,0.0
4,0,4,0.000050,0.000450,17.410068,0.025774,5.535056,0.0


# fig, ax = plt.subplots()

values = {
    'width' : 3*dp,
    'headwidth' : 3,
    'headlength' : 5
}

ax.quiver(dp*groups['i'], dp*groups['j'], groups['vx'], groups['vy'], groups['vx'], **values);

In [60]:
U,V = np.zeros((groups['j'].max()-groups['j'].min(),groups['i'].max()-groups['i'].min()), dtype = float), np.zeros((groups['j'].max()-groups['j'].min(),groups['i'].max()-groups['i'].min()), dtype = float)
X, Y = np.arange(groups['i'].min(), groups['i'].max()),  np.arange(groups['j'].min(), groups['j'].max())

for x in X:
    for y in Y:
        row = groups.loc[(groups['i'] == x) &  (groups['j'] == y)]
        try :
            U[y,x] = row['vx']
            V[y,x] = row['vy']
        except ValueError as e:
            pass

In [61]:
print(U.shape, V.shape)
print(X.shape, Y.shape)
speed = np.sqrt(U**2+V**2)
lw = 2 * speed / (speed.max()+1e-8)

(49, 179) (49, 179)
(179,) (49,)


In [62]:
fig, ax = plt.subplots(constrained_layout=True)
# ax.axis('equal')
h = ax.streamplot(x=dp*X,y=dp*Y,u=U,v=V, density = 3, color=U, linewidth=lw, cmap='plasma'); 
cbar = fig.colorbar(h.lines, ax=ax)
analysis.set_axis(ax, x = 'x', y = 'y');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Out particles plot

In [31]:
df_out_particles.head()

,x,y,vx,vy,vz,species
1,-6.607005e-07,0.004157,-184.709686,-194.369511,-136.412644,0.0
1,-5.150588e-07,0.002332,-134.390922,-44.034838,-57.670025,0.0
1,-1.003467e-06,0.002065,-242.298289,-118.385751,57.350139,0.0
1,-8.160556e-07,0.000229,-286.742703,92.057504,-31.955781,0.0
1,1.800273e-02,0.002363,376.732753,-2.923945,-60.245838,0.0


In [32]:
df_out_particles = df_out_particles.loc[df_out_particles['vx']>0]

In [33]:
# df_out_particles['species'] = df_out_particles.apply(convert_to_species, axis = 1)

In [34]:
df_out_particles['v'] = np.sqrt(df_out_particles['vx']**2+df_out_particles['vy']**2+df_out_particles['vz']**2)
df_out_particles['energy'] = df_out_particles['v']**2
df_out_particles['angle'] = np.arctan(df_out_particles['vy']/df_out_particles['vx'])

In [35]:
ions_out = df_out_particles.loc[df_out_particles['species'] == 1.0]
neutral_out =  df_out_particles.loc[df_out_particles['species'] == 0.0]

In [36]:
results = df_out_particles[['species', 'x']].groupby('species').count();
try:
    print('Out particles neutralization percentage : {:.4} %'.format(100*results.iloc[0,0]/(results.iloc[0,0]+results.iloc[1,0])))
except:
    print('100% neutralization')
    print(results)

100% neutralization
             x
species       
0.0      11370


In [37]:
vel_results = df_out_particles[['species', 'vx', 'vy', 'vz', 'v','energy']].groupby('species').agg('mean');
vel_results

,vx,vy,vz,v,energy
species,,,,,
0.0,1258.939111,10.346157,-0.235449,1376.538113,2.406150e+07


In [38]:
analysis.set_fontsizes(SMALL_SIZE = 10, MEDIUM_SIZE = 12, BIGGER_SIZE = 14)

In [39]:
fig, ax = plt.subplots(constrained_layout = True)
sns.despine(fig)

sns.histplot(
    df_out_particles.loc[df_out_particles['vx']>0], # problem : some of them are actually exciting trough the in-flux wall
    x="vx", hue="species",
    bins = 50,
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=[False,True],
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
fig, ax = plt.subplots(constrained_layout = True)
#sns.despine(fig)
ax.set_xticks([-np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2]);
ax.set_xticklabels(['-π/2', '-π/4', '0', 'π/4', 'π/2'])
sns.histplot(
    df_out_particles,
    # bins = 100,
    x="angle", hue="species",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='angle', ylabel='Count'>

### Thrust due to ions

In [41]:
df_out_particles.head()

,x,y,vx,vy,vz,species,v,energy,angle
1,0.018003,2.362874e-03,376.732753,-2.923945,-60.245838,0.0,381.530704,145565.677762,-0.007761
1,0.017610,-3.675210e-07,44.798561,-281.923287,-75.787354,0.0,295.349579,87231.373661,-1.413211
1,0.018000,5.064657e-04,72.153562,-53.137515,-244.808366,0.0,260.693053,67960.867994,-0.634773
1,0.018001,3.722523e-03,205.830924,71.136528,-56.019810,0.0,224.866614,50564.993925,0.332756
1,0.018000,2.901596e-03,83.481394,3.296491,-119.414235,0.0,145.738703,21239.769632,0.039467


In [42]:
df_out_particles_it = df_out_particles.loc[np.isin(df_out_particles.index, frames)]
duration = (np.max(frames)-np.min(frames))*dt
print(duration)

1.595e-05


In [43]:
print('Mean speed out : {:.3e} m/s.'.format(df_out_particles.mean()['vx']))

Mean speed out : 1.259e+03 m/s.


In [44]:
df_out_particles_it.sum()['vx']

2352723.3229656033

In [45]:
density_ion = 3.2e17
v_mean_injection_ion = 2000 # m/s
flux_ion = 0.25*density_ion*v_mean_injection_ion

density = 3.2e19
v_mean_injection = 200 # m/s
flux = 0.25*density*v_mean_injection

# in this simulation
mean_speed_ion = df_out_particles.loc[df_out_particles['vx']>1000].mean()['vx'] # the thing is that now I cant make the difference between I- and I (as I inject both) - I choose the criteria speed > 1000 but not too sure about that
mean_speed = df_out_particles.loc[df_out_particles['vx']<1000].mean()['vx'] # m/s
surfasic_thrust_ion = mean_speed_ion*mass*flux_ion
surfasic_thrust = mean_speed*mass*flux # total
total_thrust = surfasic_thrust + surfasic_thrust_ion
print('Ratio ion / neutral : {:.3}'.format(surfasic_thrust_ion/surfasic_thrust))
print('Thrust per injection surface : {:.2e} N/m²'.format(total_thrust))

Ratio ion / neutral : 11.0
Thrust per injection surface : 7.61e-01 N/m²


In [46]:
surface_pegase = 5e-2 * 4e-2 # m²
print('Pegase thrust : {:.3e} N'.format(total_thrust*surface_pegase))

Pegase thrust : 1.521e-03 N


In [47]:
injection_surface = 1e-3*5e-3
print('Thrust : {:.3e} N'.format(mr*df_out_particles_it.sum()['vx']*mass/(duration*injection_surface)))

Thrust : 2.005e-02 N


In [48]:
V_accel = 300 # V
e = 1.6e-19
print('Isp : {:.3e} s-1'.format(mean_speed_ion/9.81))
print('Theoretical Isp : {:.3e} s-1'.format((np.sqrt(2*e*V_accel/mass))/9.81))
print('Efficiency : {:.3} %'.format(100*mean_speed_ion/np.sqrt(2*e*V_accel/mass)))

Isp : 2.092e+03 s-1
Theoretical Isp : 2.167e+03 s-1
Efficiency : 96.5 %


df_mean = df.groupby(df.index).mean()
## Temperature evolution

First - not by cell but for the whole system.

### Theory : 

The kinetic energy density $w$ is

$w = \frac{3}{2}p(\vec{r},t) + n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$ 

With $\vec{u}(\vec{r},t) = ((<(v_x, v_y, v_z)>)(\vec{r},t))^T$ thus : $|\vec{u}(\vec{r},t)| = |<\vec{v}>|$.

Here the drift is not zero. So we have to remove it to have only $\frac{3}{2}p(\vec{r},t) = w - n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$.

This yields : 

$\frac{3}{2}p(\vec{r},t) = n(\vec{r},t)  <\frac{1}{2}mv^2> - n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$

Which yields : 

$$ \frac{3}{2}p(\vec{r},t) = \frac{m n(\vec{r},t)}{2}  (<v^2> - |<\vec{v}>|^2) $$

With $p = nkT$.

Intuition : total energy - energy contains in the drift.

In [53]:
df['v'] = np.sqrt(df['vx']*df['vx']+df['vy']*df['vy']+df['vz']*df['vz']) # df.apply(analysis.speed_norm, axis = 1)
df['v2'] = df['v']**2
df_mean = df.groupby(df.index).mean()
df_mean['v_drift'] = np.sqrt(df_mean['vx']*df_mean['vx']+df_mean['vy']*df_mean['vy']+df_mean['vz']*df_mean['vz'])

In [54]:
# maxwellian temperature 
temperature_mb = np.pi*mass*df_mean['v']**2/(8*physics.BOLTZMAN_CONSTANT)

In [55]:
df_mean['temperature'] =  mass/(3.*physics.BOLTZMAN_CONSTANT) * (df_mean['v2']-df_mean['v_drift']**2)
temperatures = df_mean['temperature']

In [56]:
fig, ax = plt.subplots(1,2)
ax[0].plot(unique_index*dt,temperatures)
ax[1].plot(unique_index*dt,temperature_mb)

analysis.set_axis(ax[0], x = 'time', y = 'temperature')
analysis.set_axis(ax[1], x = 'time', y = 'temperature')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,temperatures)
analysis.set_axis(ax, x = 'time', y = 'temperature')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt, physics.pressure_torr(nb_parts*mr[0], volume, temperature_mb)*1e3)
analysis.set_axis(ax, x = 'iteration', y = None)
ax.set_ylabel('pressure (mTorr)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: 'int' object is not subscriptable

In [59]:
sound_speed = physics.speed_of_sound(molecular_mass, temperature_mb, gamma)

In [42]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,sound_speed)
analysis.set_axis(ax, x = 'time', y = 'sound speed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Flow speed

In [43]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,df_mean['v_drift'])
analysis.set_axis(ax, x = 'time', y = 'v');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Mach number

In [44]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,df_mean['v_drift']/sound_speed)
analysis.set_axis(ax, x = 'time')
ax.set_ylabel('Mach number');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Collision with walls - analysis

In [49]:
print('Collisions with walls : {:.3e}'.format(np.sum(collisions_with_walls)*mr))

Collisions with walls : 9.741e+11


In [50]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(collisions_with_walls.index*dt, mr*collisions_with_walls); # *np.mean(mr)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## DSMC collisions analysis

### Particles collisions analysis

In [51]:
# start by summing over all cells
collisions = np.sum(collisions_per_cell, axis = (0,1))
print('Total number of collision in the simulation (in reality) : {:e} ({:e})'.format(int(np.sum(collisions_per_cell)),mr*np.sum(collisions_per_cell)));

Total number of collision in the simulation (in reality) : 1.017700e+04 (3.256640e+10)


In [52]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(index_collisions*dt, collisions*mr);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Rate between the two

In [53]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time')
ax.set_ylabel('rate')
ax.plot(index_collisions*dt, collisions_with_walls/collisions);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Mean free path 
The formula for the mean free path is : $\lambda = \frac{v_{mean}}{\nu}$, where : 
* $v_{mean}$ is the mean velocity ;
* $\nu$ is the frequency of collision for ONE particle.

We have : $\frac{1}{\nu} = \frac{N_c dt}{2 N}$ with $N_c$ the number of particle in the cell, and $N$ the number of collisions.

This yields :  $\lambda = \frac{v_{mean}N_c dt}{2N}$. There is no need for $m_r$ here as both $N_c$ and $N$ would require the multiplication to have the current number in the system.

We start by doing it for the whole system.

In [63]:
x_res = 10 # for example the number of cells during the simulations (if you want a plot per cell)
y_res = 1
x_step = 0.001 # size of the cell
y_step = 0.001

In [64]:
v_mean = df_mean['v']

In [65]:
# selecting the time iteration that we also saved for the collisions (saving frequency is not necessary the same)
v_mean = v_mean.loc[np.isin(v_mean.index,index_collisions)]
nb_particles_in_system = df['x'].groupby(df.index).agg('count').loc[np.isin(unique_index,index_collisions)]

In [66]:
mfp = physics.mean_free_path_simu(nb_particles_in_system, collisions, dt, v_mean)

### Mean free path evolution

In [67]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp)
analysis.set_axis(ax, x = 'time', y = 'mean free path')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Knudsen evolution

In [68]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp/cell_size)
analysis.set_axis(ax, x = 'time', y = 'Knudsen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Total mean free path

Accouting for collisions between particles and **collisions with walls**.

In the case where the number of collisions with walls is much bigger than the number of collisions between particles, then this mean free path is basically the average distance a particle covers between two wall collisions.

In [69]:
mfp_ = 0.5*nb_particles_in_system*dt*v_mean/(collisions+collisions_with_walls)

#### Total Mean free path evolution 

In [70]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp_)
analysis.set_axis(ax, x = 'time', y = 'mean free path');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Total Knudsen - evolution

In [71]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp_/cell_size)
analysis.set_axis(ax, x = 'time', y = 'Knudsen');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Out particle analysis

In [54]:
fig, ax = plt.subplots()
mass_flow_rate = mr*physics.compute_mass_flow_rate(total_deleted, dt, mass)
analysis.set_axis(ax, x = 'time', y = 'mass flow rate')
plt.plot(mass_flow_rate.index*dt, mass_flow_rate);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean number of particles

In [55]:
averages = averages_per_cell.groupby(averages_per_cell.index).sum() # returns a pandas Series

In [56]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(averages.index*dt, averages);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean acceptance probability

In [57]:
proba = total_proba/collisions
print('Mean proba : {:e}'.format(np.mean(proba)))
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'probability')
ax.plot(proba.index*dt, proba, label = 'proba');

Mean proba : 2.095243e-02


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Maximum probability

In [58]:
pmax = pmax_per_cell.groupby(pmax_per_cell.index).sum() # returns a pandas Series

In [59]:
np.max(pmax_per_cell)/np.min(pmax_per_cell)

25.948142853273396

In [60]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'probability')
ax.plot(pmax.index*dt, pmax);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean distance between two colliding particles

In [61]:
fig, ax = plt.subplots()
dist = total_distance/collisions
print('Mean distance : {:e} m'.format(np.mean(dist)))
analysis.set_axis(ax, x = 'time', y = 'distance')
ax.plot(dist.index*dt, dist);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean distance : 6.338776e-05 m


### Velocity axis distribution per *cell*

In [ ]:
# spatial_hist2d plots an hist per cell defined by the params below
# it is a 2D plot of size y_res * x_res
# where each plot (i,j) takes the particle between i*x_step < x < (i+1)*x_step and j*y_step < y < (j+1)*y_step
# for i varying between between 0 and x_res-1, and j varying between between 0 and y_res-1
# You should not use too many cells because it quickly becomes very hard to view.
# Params
x_res = 11 # for example the number of cells during the simulations (if you want a plot per cell)
y_res = 5
x_step = 0.001 # size of the cell
y_step = 0.001 

# Plotting - vx
# fig, axes = analysis.spatial_hist2d(df, frames, val = 'vx', x_res = x_res, y_res = y_res, x_step = x_step, y_step=y_step);

In [ ]:
# Plotting - vy
# fig, axes = analysis.spatial_hist2d(df, frames, val = 'vy', x_res = x_res, y_res = y_res, x_step = x_step, y_step=y_step);

## Bonus - Not necessarily recommended and takes lots of time
Next is the plot of the system for each saved step. 

In [ ]:
# creating speed_norm (for the color)
# df['speed_norm'] = np.sqrt(df['vx']**2+df['vy']**2+df['vz']**2) -> DO NOT NEED THAT

In [ ]:
%matplotlib notebook
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

def update_hist(num, df):
    dfit = df.loc[df.index == num]

    scat.set_offsets(np.c_[dfit['x'],dfit['y']])
    scat.set_array(df['v'])
    
    ax.set_title('{}/{}'.format(num+1, iterations), fontsize=15)

fig, ax = plt.subplots()
dfit = df.loc[df.index == 0]
scat = ax.scatter(dfit['x'], dfit['y'], s=0.1, c = dfit['v'], cmap='seismic') #  c = df['speed_norm']
analysis.set_axis(ax, x = 'x', y = 'y')
# if you want the boundaries to be plotted and the grid too, you have to initialize those fields yourself.
# in the other case, only particles will be plotted.
# plot_boundaries(ax, segments)
# plot_grid(ax, resolutions, system_shape)

ax.set_title('{}/{}'.format(1, iterations), fontsize=12)
ax.axis('equal')

interval = 40 # 25 images per second
anim = FuncAnimation(fig, update_hist, interval=interval, frames=iterations, fargs=(df, ), save_count=iterations)
anim.save(dir_path/'system_state_evo.avi', dpi = 300);